## Implementation of CNN from SCRATCH

In [1]:
import numpy as np
import math

In [2]:
class Conv2d:
    id_ = 0
    def __init__(self,input_x:np.ndarray,filter_shape:tuple,number_of_filters:int,padding:int=1,
                 stride:int=1,max_pool_shape:tuple=None,max_pool_stride:int=2):
        self.input_x = input_x # (channel,H,W) # X
        self.filter_shape = filter_shape # (F_h,F_W)
        self.number_of_filters = number_of_filters
        self.padding = padding
        self.stride = stride
        self.filters = None # W
        self.max_pool_shape = max_pool_shape
        self.max_pool_stride = max_pool_stride    
        self.max_pool_arg_max_indices = []
        self.max_pool_dH= None
        self.id_ = Conv2d.id_
        Conv2d.id_ += 1
        
    def xavier_weight_init(self,shape:tuple,fan_in:int,fan_out:int) -> np.ndarray:
        return np.random.randn(*shape)*np.sqrt(2/(fan_in+fan_out))
        
    # forward
    def __call__(self,is_predicting=False) -> np.ndarray:
        
        # Calculating output Dimensions
        output_h = math.floor((((self.input_x.shape[1]+2*self.padding-self.filter_shape[0]) /self.stride) + 1 ))
        output_w = math.floor((((self.input_x.shape[2]+2*self.padding-self.filter_shape[1]) /self.stride) + 1 ))
        
        
        # Padding
        if self.padding != 0:
            ch_list = [np.pad(self.input_x[x],self.padding) for x in range(self.input_x.shape[0])]
            self.input_x = np.vstack([[ch] for ch in ch_list])
        

        # Initializing output
        output = np.zeros((self.number_of_filters,output_h,output_w)) # (number_of_filters,H,W)
        
        # Initializing filters
        if not is_predicting:
            self.filters = [self.xavier_weight_init(
                shape=(self.input_x.shape[0],self.filter_shape[0],self.filter_shape[1]),
                fan_in = self.input_x.shape[0] * self.input_x.shape[1] * self.input_x.shape[2],
                fan_out = self.filter_shape[0] * self.filter_shape[1] * self.input_x.shape[0] if self.max_pool_shape is None else self.filter_shape[0] * self.filter_shape[1] * self.input_x.shape[0] /(self.max_pool_shape[0]*self.max_pool_shape[1])  
            ) for _ in range(self.number_of_filters)]

        # Convolving
        for index, filter_ in enumerate(self.filters):
            out = np.empty((output_h,output_w))
            row_counter = 0
            for row in np.arange(0,self.input_x.shape[1],step=self.stride):
                col_counter = 0
                if row+self.filter_shape[0] > self.input_x.shape[1] or row > self.input_x.shape[1]:
                    pass
                else:
                    for col in np.arange(0,self.input_x.shape[2],step=self.stride):
                        if col+self.filter_shape[1] >  self.input_x.shape[2] or col > self.input_x.shape[2]:
                            pass
                        else:
                            out[row_counter,col_counter] =(self.input_x[:,row:row+self.filter_shape[0],col:col+self.filter_shape[1]] * filter_).sum()
                            col_counter += 1
                    
                    row_counter += 1
            output[index] = out
        
        
        # Max Pooling       
        if self.max_pool_shape is not None:
            self.max_pool_dH = np.zeros(output.shape)
            pooling_output = np.zeros((output.shape[0],
                                       ((output.shape[1]-self.max_pool_shape[0])//self.max_pool_stride)+1,
                                       ((output.shape[2]-self.max_pool_shape[1])//self.max_pool_stride)+1))
            for i in range(output.shape[0]):
                row_counter = 0
                indices = []
                for row in np.arange(0,self.input_x.shape[1],step=self.max_pool_stride):
                    col_counter = 0
                    if row+self.max_pool_shape[0] > output.shape[1]:
                        pass
                    else:
                        for col in np.arange(0,self.input_x.shape[2],step=self.max_pool_stride):
                            if col+self.max_pool_shape[1] > output.shape[2]:
                                pass
                            else:
                                result = output[i,row:row+self.max_pool_shape[0],col:col+self.max_pool_shape[1]]
                                argmax_row,argmax_col = np.unravel_index(result.argmax(), result.shape)
                                indices.append((argmax_row,argmax_col))
                                pooling_output[i,row_counter,col_counter] = result[argmax_row,argmax_col]
                                col_counter += 1 
                        row_counter += 1
                self.indices.append(indices)
            return pooling_output
        
        return output  
    
    def backward(self,dH):
        
        (n_C,n_H, n_W) = dH.shape
        
        if self.max_pool_shape is not None:
            #Expanding dh 
            for i in range(self.number_of_filters):
                index=0
                row_counter = 0
                for row in np.arange(0,self.max_pool_dH.shape[1],step=self.max_pool_stride):
                    if row+self.max_pool_shape[0] > self.max_pool_dH.shape[1]:
                        pass
                    col_counter = 0
                    for col in np.arange(0,self.max_pool_dH.shape[2],step=self.max_pool_stride):
                        if col+self.max_pool_shape[1] > self.max_pool_dH.shape[2]:
                                pass
                        self.max_pool_dH[i,row:row+self.max_pool_shape[0],col:col+self.max_pool_shape[1]][self.max_pool_arg_max_indices[i][index][0],self.max_pool_arg_max_indices[i][index][1]] = dH[i,row_counter,col_counter]
                        index+=1
                        col_counter+=1
                    row_counter+=1
            dH = self.max_pool_dH
                    
        dx = np.zeros(self.input_x.shape)
        dw = np.zeros(n_C,self.input_x.shape[0],self.filter_shape[0],self.filter_shape[1]) # filter_numbers , channel_numbers , H . W
        
        
        for index in range(n_C):
            dh = np.vstack([[dH[i]] for _ in range(self.input_x.shape[0])])
            conv_out = np.empty((self.input_x.shape[0],self.filter_shape[0],self.filter_shape[1]))
            
            row_counter = 0
            for row in np.arange(0,self.input_x.shape[1],step=self.stride):
                col_counter = 0
                if row+n_H > self.input_x.shape[1] or row > self.input_x.shape[1]:
                    pass
                else:
                    for col in np.arange(0,self.input_x.shape[2],step=self.stride):
                        if col+n_W >  self.input_x.shape[2] or col > self.input_x.shape[2]:
                            pass
                        else:
                            conv_out[row_counter,col_counter] =(self.input_x[:,row:row+n_H,col:col+n_W] * dh).sum()
                            col_counter += 1
                    
                    row_counter += 1
            dw[index] = conv_out
            
            
            
        
        
        
        return dx, dw
        
        
        

In [4]:
A = np.arange(225).reshape((9,5,5))

In [5]:
A.shape

(9, 5, 5)

In [6]:
conv2d = Conv2d(input_x=A,filter_shape=(1,1),number_of_filters=1,padding=1,stride=2)

In [7]:
y = conv2d(is_predicting=False)

In [8]:
y.shape

(1, 4, 4)

In [9]:
conv2d_2 = Conv2d(input_x=y,filter_shape=(2,2),number_of_filters=3,padding=2,stride=2)

In [10]:
yy = conv2d_2()
yy.shape

(3, 4, 4)

In [12]:
conv2d.backward(2)

(9, 7, 7)


TypeError: data type not understood

In [17]:
a = np.array([[4,2],[5,1]])
a

array([[4, 2],
       [5, 1]])

In [18]:
np.vstack([[a],[a],[a]])

array([[[4, 2],
        [5, 1]],

       [[4, 2],
        [5, 1]],

       [[4, 2],
        [5, 1]]])

In [19]:
np.unravel_index(a.argmax(), a.shape)

(1, 0)